<a href="https://colab.research.google.com/github/jyryu3161/lec_docking/blob/main/Docking_%EC%8B%A4%EC%8A%B5_multiple_molecules.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Molecular Docking 실습

In [1]:
# download & extract
url = 'https://anaconda.org/rdkit/rdkit/2018.09.1.0/download/linux-64/rdkit-2018.09.1.0-py36h71b666b_1.tar.bz2'
!curl -L $url | tar xj lib
# move to python packages directory
!mv lib/python3.6/site-packages/rdkit /usr/local/lib/python3.6/dist-packages/
x86 = '/usr/lib/x86_64-linux-gnu'
!mv lib/*.so.* $x86/
# rdkit need libboost_python3.so.1.65.1
!ln -s $x86/libboost_python3-py36.so.1.65.1 $x86/libboost_python3.so.1.65.1

!git clone https://github.com/jyryu3161/lec_docking
!chmod u+x ./lec_docking/qvina2.1

!pip install --pre deepchem
!pip install ogb
!pip install py3Dmol # 3D Molecular Visualizer
!pip install oddt
!pip install -U ProDy
!apt-get install pymol 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3939    0  3939    0     0   3449      0 --:--:--  0:00:01 --:--:--  3449
100 20.2M  100 20.2M    0     0  3311k      0  0:00:06  0:00:06 --:--:-- 5330k
Cloning into 'lec_docking'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 61 (delta 31), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (61/61), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 693 kB 37.9 MB/s 
     |████████████████████████████████| 29.3 MB 94.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 78 kB 6.5 MB/s 
  Created wheel for littleutils: filename=littl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 16.9 MB/s 
     |████████████████████████████████| 3.1 MB 53.9 MB/s 
  Created wheel for ProDy: filename=ProDy-2.3.1-cp38-cp38-linux_x86_64.whl size=11750348 sha256=a3d0de4a607f5388ce605ceb46b3dca52d6f13ca4e5b0fbdfde6b94458d02512
  Stored in directory: /root/.cache/pip/wheels/fe/70/79/df801d96836c4ab4d3850a6bba7370cc21a66599b5bc05b616
Successfully built ProDy
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  apbs freeglut3 libglew2.0 libmaloc1 pymol-data python-opengl python-pmw
Suggested packages:
  glew-utils libgle3 python-pmw-doc
The following NEW packages will be installed:
  apbs freeglut3 libglew2.0 lib

# Input setting

In [2]:
protein_pdb_file = '6m0k_processed.pdb'
center = (-10.5, 15.6, 68.7) # 예측된 binding site를 넣을 것 - x, y, z 

input_smiles_file = './input_smiles.txt'
docking_output_dir = './output/'

# Run molecular docking

In [6]:
import py3Dmol
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
import oddt
from oddt.docking.AutodockVina import autodock_vina
from oddt.docking import AutodockVina
import os
import glob
from prody import *
import os

def make_3d_coordinate(m):
    target_mols = []
    m = Chem.AddHs(m)
    potential = AllChem.ETKDG()
    AllChem.EmbedMolecule(m, potential)
    cids = AllChem.EmbedMultipleConfs(m, numConfs=1, numThreads=15)
    
    for conf in m.GetConformers():
        tm = Chem.Mol(m,False,conf.GetId())
        res = AllChem.MMFFOptimizeMoleculeConfs(tm)
        target_mols.append(tm)
    return target_mols[0]

def drawit2(m,confId=-1):
    mb = Chem.MolToMolBlock(m,confId=confId)
    p = py3Dmol.view(width=400, height=400)
    p.addModel(mb,'sdf')
    p.setStyle({'stick':{}})
    p.setBackgroundColor('0xeeeeee')
    p.zoomTo()
    return p

def DrawComplex(protein,ligand):
    complex_pl = Chem.MolToPDBBlock(Chem.CombineMols(protein,ligand))
    view = py3Dmol.view(width=600,height=600)
    view.addModel(complex_pl,'pdb')
    chA = {'chain':['H','L','I']}
    chB = {'resn':'UNL'}
    view.setStyle(chA,{'cartoon': {'color':'spectrum'}})
    #view.setStyle(chA,{'lines': {}})
    view.addSurface(py3Dmol.VDW,{'opacity':0.8}, chB)
    view.setStyle(chB,{'stick':{}})
    view.zoomTo()
    return view   

tmp_mol_dir = docking_output_dir+'/molecules/'
tmp_output_dir = docking_output_dir+'/docking_results/'

try:
    os.mkdir(docking_output_dir)
except:
    pass

try:
    os.mkdir(tmp_mol_dir)
except:
    pass

try:
    os.mkdir(tmp_output_dir)
except:
    pass

with open(input_smiles_file, 'r') as fp:
    for line in fp:
        sptlist = line.strip().split('\t')
        compound_id = sptlist[0].strip()
        smiles = sptlist[1].strip()

        mol = Chem.MolFromSmiles(smiles)    
        mol = Chem.AddHs(mol)
        mol = make_3d_coordinate(mol)

        with Chem.SDWriter(tmp_mol_dir+'%s.sdf'%(compound_id)) as w:
            w.write(mol)


protein_file = protein_pdb_file

protein = next(oddt.toolkit.readfile('pdb', protein_file))
protein.protein = True

protein.addh(only_polar=True)

files = glob.glob(tmp_mol_dir+'/*.sdf')
for each_file in files:
    basename = os.path.basename(each_file).split('.')[0].strip()
    mol = Chem.MolFromMolFile(each_file)

    mol2 = Chem.MolToMolBlock(mol)
    dock_mol = oddt.toolkit.readstring('sdf', mol2)

    vina_obj = autodock_vina(protein=protein, center=center, size=(20, 20, 20), num_modes=1, exhaustiveness=8, executable='./lec_docking/qvina2.1')

    docking_outputs = vina_obj.dock([dock_mol])
    for each_output in docking_outputs:
        vina_affinity = each_output.data['vina_affinity']
        AutodockVina.write_vina_pdbqt(each_output, tmp_output_dir, name_id='best_docking_pose_%s'%(basename))
        
        print ("COMPOUND:%s, binding energy (kcal/mol) : %s"%(basename, vina_affinity))
        break




COMPOUND:CNP0106606, binding energy (kcal/mol) : -9.1
COMPOUND:CNP0297651, binding energy (kcal/mol) : -7.2
COMPOUND:CNP0369807, binding energy (kcal/mol) : -6.0
COMPOUND:CNP0115481, binding energy (kcal/mol) : -6.7
COMPOUND:CNP0115074, binding energy (kcal/mol) : -6.3


KeyboardInterrupt: ignored